In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os
import glob
import cv2

Crop images



In [4]:
IMG_DIR = 'Data/images'
TXT_DIR = 'Data/labels'
OUT_DIR = 'Data/cropped'

def adjust(value, m):
    return int(min(max(value, 0), m))

os.makedirs(OUT_DIR, exist_ok=True)
# loop every image
for img_path in glob.glob(os.path.join(IMG_DIR, '*.*')):
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(TXT_DIR, base + '.txt')
    # print(base)
    # print(txt_path)
    if not os.path.isfile(txt_path):
        print(f'[WARN] cannot find txt:{txt_path}')
        continue

    # get coordinate
    # format: <label> <x_center> <y_center> <width> <height>
    with open(txt_path, 'r') as f:
        line = f.readline().strip()
    coords = [float(v) for v in line.split()]

    _, x_center, y_center,  width_frac, height_frac = coords
    x_center, y_center,  width_frac, height_frac = map(float, 
            (x_center, y_center,  width_frac, height_frac))
    
    img = cv2.imread(img_path)
    H, W = img.shape[:2]
    
    x_center = int(x_center * W)
    y_center = int(y_center * H)
    width = int(width_frac * W) 
    height = int(height_frac * H)

    left_x = int(max(x_center - width / 2, 0))
    left_y = int(min(y_center - height / 2, W))

    cropped = img[left_y:left_y + height, left_x:left_x + width]
    
    out_path = os.path.join(OUT_DIR, base + '.png')
    success = cv2.imwrite(out_path, cropped)
    print(f'Cropped {base} → {out_path}')


Cropped cdaa9c9f-5b67582e96e8ca90f74b493cdf15f40 → Data/cropped/cdaa9c9f-5b67582e96e8ca90f74b493cdf15f40.png
Cropped a74f55ce-20250510_183827 → Data/cropped/a74f55ce-20250510_183827.png
Cropped 55ba4e80-fce64740-IMG_20250427_165448 → Data/cropped/55ba4e80-fce64740-IMG_20250427_165448.png
Cropped 22e3fd33-IMG_20250503_134158 → Data/cropped/22e3fd33-IMG_20250503_134158.png
Cropped 1b30ad36-acad2ddd8f04a9227d348fb8d41f296 → Data/cropped/1b30ad36-acad2ddd8f04a9227d348fb8d41f296.png
Cropped 2fc49ea7-527b54a3b8069779ff1a206c0d313ac → Data/cropped/2fc49ea7-527b54a3b8069779ff1a206c0d313ac.png
Cropped 6a9156cb-93249830-IMG_20250427_151945 → Data/cropped/6a9156cb-93249830-IMG_20250427_151945.png
Cropped 222df708-894876bf-IMG_20250427_165035 → Data/cropped/222df708-894876bf-IMG_20250427_165035.png
Cropped 6f508bfd-668443bc-IMG_20250427_152235 → Data/cropped/6f508bfd-668443bc-IMG_20250427_152235.png
Cropped 5fe4dbc0-eb520b4d7ff9b04f70b7ca2c0f94ede → Data/cropped/5fe4dbc0-eb520b4d7ff9b04f70b7ca2c0f

In [21]:
import os, glob

IMG_DIR = 'Data/images'
TXT_DIR = 'Data/labels'

# 收集所有 image/.txt 的前缀
img_bases = {os.path.splitext(os.path.basename(p))[0]
             for p in glob.glob(os.path.join(IMG_DIR, '*.*'))}
txt_bases = {os.path.splitext(os.path.basename(p))[0]
             for p in glob.glob(os.path.join(TXT_DIR, '*.txt'))}

# 哪些 image 没有对应的 txt？
print('Images without txt:', sorted(img_bases - txt_bases))
# 哪些 txt 没有对应的 image？
print('Txts without image:', sorted(txt_bases - img_bases))


Images without txt: []
Txts without image: []


In [ ]:
# 1. DataLoader (no normalization! we’re computing it)
ds = datasets.ImageFolder(
    "path/to/images",
    transform=transforms.ToTensor()        # yields [0,1] float32 tensor
)
loader = DataLoader(ds, batch_size=64, num_workers=8, pin_memory=True)

# 2. Accumulators
sum_c   = torch.zeros(3)   # sum per channel
sum_sq  = torch.zeros(3)   # sum of squares per channel
n_pix   = 0                # total number of pixels per channel
sum_all = 0.0              # sum over all pixels & channels

for imgs, _ in loader:
    # imgs.shape == (B, C, H, W)
    B, C, H, W = imgs.shape
    n_pix += B * H * W

    # a) per‐channel sums
    sum_c   += imgs.sum(dim=[0,2,3])           # shape (C,)
    sum_sq  += (imgs*imgs).sum(dim=[0,2,3])    # shape (C,)

    # b) overall sum
    sum_all += imgs.sum()                      # scalar

# 3. Compute means & stds
#  — Per‐channel
mean_c = sum_c   / n_pix                       # shape (C,)
var_c  = sum_sq  / n_pix - mean_c**2
std_c  = torch.sqrt(var_c)

#  — Global image mean (scalar)
#    total pixels×channels = n_pix * C
global_mean = sum_all / (n_pix * C)

print("Per-channel mean:", mean_c)
print("Per-channel std: ", std_c)
print("Global image mean (all pixels & channels):", global_mean)
